In [ ]:
from google.colab import drive

# Attempt to mount Google Drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print("Error mounting Google Drive:", str(e))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


In [ ]:
import cv2
import torch
import requests
import numpy as np
from PIL import Image
from transformers import AutoProcessor, RTDetrForObjectDetection, VitPoseForPoseEstimation

try:
    from ultralytics import YOLO
    print("YOLO already installed!")
except ImportError:
    print("Installing YOLO...")
    !pip install ultralytics

YOLO already installed!


Model 1: ViTPose

In [ ]:
class PoseEstimator_ViTPose:
    def __init__(self, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.person_image_processor = AutoProcessor.from_pretrained("PekingU/rtdetr_r50vd_coco_o365")
        self.person_model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd_coco_o365", device_map=self.device)
        self.pose_image_processor = AutoProcessor.from_pretrained("usyd-community/vitpose-base-simple")
        self.pose_model = VitPoseForPoseEstimation.from_pretrained("usyd-community/vitpose-base-simple", device_map=self.device)

    # For the load_image method, it does not have to input an url, as long as it returns an image object
    def load_image(self, fpath):
        return Image.open(fpath)

    def detect_humans(self, image):
        inputs = self.person_image_processor(images=image, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.person_model(**inputs)
        results = self.person_image_processor.post_process_object_detection(
            outputs, target_sizes=torch.tensor([(image.height, image.width)]), threshold=0.3
        )
        result = results[0]  # take first image results
        person_boxes = result["boxes"][result["labels"] == 0].cpu().numpy()
        person_boxes[:, 2] = person_boxes[:, 2] - person_boxes[:, 0]
        person_boxes[:, 3] = person_boxes[:, 3] - person_boxes[:, 1]
        return person_boxes

    def detect_keypoints(self, image, person_boxes):
        inputs = self.pose_image_processor(image, boxes=[person_boxes], return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.pose_model(**inputs)
        pose_results = self.pose_image_processor.post_process_pose_estimation(outputs, boxes=[person_boxes])
        return pose_results[0]  # results for first image

    def estimate_pose(self, fpath):
        image = self.load_image(fpath)
        person_boxes = self.detect_humans(image)
        keypoints = self.detect_keypoints(image, person_boxes)
        return keypoints

Model 2: YOLO

In [ ]:
class PoseEstimator_YOLOv8:
    def __init__(self, model_path, device="cuda:0"):
        self.model = YOLO(model_path)
        self.device = device

    def train(self, data_path, epochs=100, img_size=640):
        train_results = self.model.train(
            data=data_path,
            epochs=epochs,
            imgsz=img_size,
            device=self.device
        )
        return train_results

    def evaluate(self):
        metrics = self.model.val()
        return metrics

    def detect(self, image_path):
        results = self.model(image_path)
        return results

    def show_results(self, results):
        results[0].show()

    def export_model(self, format="onnx"):
        path = self.model.export(format=format)
        return path

In [ ]:
def is_detected(keypoints, confidence_threshold=0.5):
    # Check if the list is empty
    if not keypoints:
        return False

    # Extract the first dictionary in the list
    keypoint_data = keypoints[0]

    # Check if 'keypoints' tensor is empty
    if keypoint_data['keypoints'].numel() == 0:
        return False

    # Check if any keypoint has a confidence score above the threshold
    scores = keypoint_data['scores']
    if scores.numel() > 0 and (scores > 0.5).any():
        return True

    return False

In [ ]:
def process_video(video_path, pose_estimator, confidence_threshold=0.5):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return False

    # Get the video's FPS
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * 1)  # Process one frame every 1 second
    print(f"Processing every {frame_interval} frames (FPS: {fps})")

    frame_count = 0  # Counter for processed frames
    total_detections = 0  # Counter for total detections

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break  # Exit the loop if no more frames are available

        # Process only frames at 0.5-second intervals
        if frame_count % frame_interval != 0:
            frame_count += 1
            continue

        # Save the frame temporarily for processing
        temp_image_path = "/content/temp_frame.jpg"
        cv2.imwrite(temp_image_path, frame)

        # Perform pose estimation on the frame
        keypoints = pose_estimator.estimate_pose(temp_image_path)

        # Check if a person is detected in this frame
        if is_detected(keypoints, confidence_threshold):
            print(f"Person detected at {frame_count / fps:.2f} seconds")
            total_detections += 1

        frame_count += 1

    # Release the video capture object
    cap.release()

    print(f"Total detections: {total_detections}")
    return total_detections > 0  # Return True if at least one detection occurred

In [ ]:
# Example usage
def main():

    '''
    # Test image path
    image_path = "/content/drive/MyDrive/FYP/test_image.jpg"

    # For YOLO model
    yolo_model = PoseEstimator_YOLOv8("yolov8n-pose.pt")
    results = yolo_model.detect(image_path)
    yolo_model.show_results(results)
    print("YOLO Keypoints:", results)

    # For ViTPose model
    pose_estimator = PoseEstimator_ViTPose()
    keypoints = pose_estimator.estimate_pose(image_path)
    print("ViTPose Keypoints:", keypoints)

    # To determine if there is a person in front of the camera
    print("Person Detected:", is_detected(keypoints))
    '''

    # Test video path
    video_path = "/content/drive/MyDrive/FYP/test_video.mp4"

    # Initialize the pose estimator (e.g., ViTPose or YOLO)
    pose_estimator = PoseEstimator_ViTPose()  # Replace with your actual pose estimator

    # Process the video
    person_detected = process_video(video_path, pose_estimator, confidence_threshold=0.5)

    # Output the final result
    print("Person detected in video:", person_detected)

if __name__ == "__main__":
    main()

Processing every 30 frames (FPS: 30.001244864932154)
Person detected at 0.00 seconds
Person detected at 1.00 seconds
Person detected at 2.00 seconds
Person detected at 3.00 seconds
Person detected at 4.00 seconds
Person detected at 5.00 seconds
Person detected at 6.00 seconds
Person detected at 7.00 seconds
Person detected at 8.00 seconds
Total detections: 9
Person detected in video: True
